#  Anomaly detection in cellular networks

## Introduction

The purpose of this notebook is to solve a anomaly detection problem proposed as a competition in the Kaggle InClass platform.

## Problem description

### Context:

Traditionally, the design of a cellular network focuses on the optimization of energy and resources that guarantees a smooth operation even during peak hours (i.e. periods with higher traffic load). 
However, this implies that cells are most of the time overprovisioned of radio resources. 
Next generation cellular networks ask for a dynamic management and configuration in order to adapt to the varying user demands in the most efficient way with regards to energy savings and utilization of frequency resources. 
If the network operator were capable of anticipating to those variations in the users’ traffic demands, a more efficient management of the scarce (and expensive) network resources would be possible.
Current research in mobile networks looks upon Machine Learning (ML) techniques to help manage those resources. 
In this case, you will explore the possibilities of ML to detect abnormal behaviors in the utilization of the network that would motivate a change in the configuration of the base station.


### Objective

The objective of the network optimization team is to analyze traces of past activity, which will be used to train an ML system capable of classifying samples of current activity as:
 - 0 (normal): current activity corresponds to normal behavior of any working day and. Therefore, no re-configuration or redistribution of resources is needed.
 - 1 (unusual): current activity slightly differs from the behavior usually observed for that time of the day (e.g. due to a strike, demonstration, sports event, etc.), which should trigger a reconfiguration of the base station.

### Dataset

The dataset has been obtained from a real LTE deployment. During two weeks, different metrics were gathered from a set of 10 base stations, each having a different number of cells, every 15 minutes. 

The dataset is provided in the form of a csv file, where each row corresponds to a sample obtained from one particular cell at a certain time. Each data example contains the following features:

 - Time : hour of the day (in the format hh:mm) when the sample was generated.
 - CellName1: text string used to uniquely identify the cell that generated the current sample. CellName is in the form xαLTE, where x identifies the base station, and α the cell within that base station (see the example in the right figure).
 - PRBUsageUL and PRBUsageDL: level of resource utilization in that cell measured as the portion of Physical Radio Blocks (PRB) that were in use (%) in the previous 15 minutes. Uplink (UL) and downlink (DL) are measured separately.
 - meanThrDL and meanThrUL: average carried traffic (in Mbps) during the past 15 minutes. Uplink (UL) and downlink (DL) are measured separately.
 - maxThrDL and maxThrUL: maximum carried traffic (in Mbps) measured in the last 15 minutes. Uplink (UL) and downlink (DL) are measured separately.
 - meanUEDL and meanUEUL: average number of user equipment (UE) devices that were simultaneously active during the last 15 minutes. Uplink (UL) and downlink (DL) are measured separately.
 - maxUEDL and maxUEUL: maximum number of user equipment (UE) devices that were simultaneously active during the last 15 minutes. Uplink (UL) and downlink (DL) are measured separately.
 - maxUE_UL+DL: maximum number of user equipment (UE) devices that were active simultaneously in the last 15 minutes, regardless of UL and DL.
 - Unusual: labels for supervised learning. A value of 0 determines that the sample corresponds to normal operation, a value of 1 identifies unusual behavior.

## Libraries

In [1]:
import os
import sys
import random
random.seed(888) #set seed for reproducibility
from zipfile import ZipFile
from IPython.display import Image


#Analysis
import pyspark
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    print('WARN: Something wrong with pyspark library. Please check configuration settings!')
    
#Feature Engineering
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, MinMaxScaler
#Model Training
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
    
# Reloads functions each time so you can edit a script and not need to restart the kernel
%load_ext autoreload
%autoreload 2

## Helpers

In [2]:
def get_root_dir(src:str, max_nest:int) -> str:
    '''
    Specify paths and appending directories with relevant python source code.
    '''
    root_dir = os.curdir
    nest = 0
    while src not in os.listdir(root_dir) and nest < max_nest:
        root_dir = os.path.join(os.pardir, root_dir)     # Look up the directory structure for a src directory
        nest += 1
        
    # If you don't find the src directory, the root directory is this directory
    root_dir = os.path.abspath(root_dir) if nest < max_nest else os.path.abspath(
    os.curdir)
    
    return root_dir

def set_src(root_dir:str, src:str) -> str:
    '''
     Get the source directory and append path to access python packages/scripts within directory
    '''
    if src in os.listdir(root_dir):
        src_dir = os.path.join(root_dir, src)
        sys.path.append(src_dir)
    return sys.path[-1]

def set_folder(root_dir:str, folder:str) -> str:
    '''
    Set the folder path based on the folder name
    '''
    folder_path = os.path.join(
        root_dir, folder) if folder in os.listdir(root_dir) else os.curdir
    return folder_path

def set_path(path:str, dirname:str) -> str:
    '''
    '''
    return os.path.join(path, dirname)

def unzip(inpath:str, outpath:str) -> None:
    zf = ZipFile(inpath, 'r')
    zf.extractall(outpath)
    zf.close()
    
def metrics(dataframe, actual, predicted):
    '''
    Calculates evaluation metrics from predicted results
    
    Input:
    ---------
        dataframe: spark.sql.dataframe with the real and predicted values [object]
        actual:  Name of column with observed target values [string]
        predicted: Name of column with predicted values [string]
        
    
    Output:
    ---------
        None
    '''
       
    # Along each row are the actual values and down each column are the predicted
    dataframe = dataframe.withColumn(actual, col(actual).cast('integer'))
    dataframe = dataframe.withColumn(predicted, col(predicted).cast('integer'))
    cm = dataframe.crosstab(actual, predicted)
    cm = cm.sort(cm.columns[0], ascending = True)
    
    # Adds missing column in case just one class was predicted
    if not '0' in cm.columns:
        cm = cm.withColumn('0', lit(0))
    if not '1' in cm.columns:
        cm = cm.withColumn('1', lit(0))
    
    # Subsets values from confusion matrix
    zero = cm.filter(cm[cm.columns[0]] == 0.0)
    first_0 = zero.take(1)
    
    one = cm.filter(cm[cm.columns[0]] == 1.0)
    first_1 = one.take(1)
    
    tn = first_0[0][1]
    fp = first_0[0][2]
    fn = first_1[0][1]
    tp = first_1[0][2]
    
    # Calculate metrics from values in the confussion matrix
    if (tp == 0):
        acc = float((tp + tn) / (tp + tn + fp + fn))
        sen = 0
        spe = float((tn) / (tn + fp))
        prec = 0
        rec = 0
        f1 = 0
    elif (tn == 0):
        acc = float((tp + tn) / (tp + tn + fp + fn))
        sen = float((tp) / (tp + fn))
        spe = 0
        prec = float((tp) / (tp + fp))
        rec = float((tp) / (tp + fn))
        f1 = 2 * float((prec * rec) / (prec + rec))
    else:
        acc = float((tp + tn) / (tp + tn + fp + fn))
        sen = float((tp) / (tp + fn))
        spe = float((tn) / (tn + fp))
        prec = float((tp) / (tp + fp))
        rec = float((tp) / (tp + fn))
        f1 = 2 * float((prec * rec) / (prec + rec))

    # Print results
    print('Confusion Matrix and Statistics: \n')
    cm.show()
    
    print('True Positives:', tp)
    print('True Negatives:', tn)
    print('False Positives:', fp)
    print('False Negatives:', fn)
    print('Total:', dataframe.count(), '\n')
    
    print('Accuracy: {0:.2f}'.format(acc))
    print('Sensitivity: {0:.2f}'.format(sen))
    print('Specificity: {0:.2f}'.format(spe))
    print('Precision: {0:.2f}'.format(prec))
    print('Recall: {0:.2f}'.format(rec))
    print('F1-score: {0:.2f}'.format(f1))
    
    # Create spark dataframe with results
    l = [(acc, sen, spe, prec, rec, f1)]
    df = spark.createDataFrame(l, ['Accuracy', 'Sensitivity', 'Specificity', 'Precision', 'Recall', 'F1'])

    return(df)

## Setup

In [26]:
root_dir = get_root_dir('src', 5)
src_dir = set_src(root_dir, 'src')
data_dir = set_folder(root_dir, 'data')
raw_data_dir = set_path(data_dir, 'raw')
interim_data_dir = set_path(data_dir, 'interim')
processed_data_dir = set_path(data_dir, 'processed')
figures_dir = set_folder(root_dir, 'figures')
features_dir = set_folder(root_dir, 'features')
index_features_dir = set_path(features_dir, 'index')
ohe_features_dir = set_path(features_dir, 'ohe')
std_features_dir = set_path(features_dir, 'std')
models_dir = set_folder(root_dir, 'models')
deliverables_path = set_folder(root_dir, 'deliverables')

# 1. Data

## Initiate Spark session

In [4]:
#If not exists create a spark session named Anomaly Detection where the master node is local
spark = SparkSession.builder \
    .master("local[4]") \
    .appName("Anomaly Detection") \
    .getOrCreate()

In [5]:
spark.getActiveSession()

## Load

### Set path

In [6]:
train_path = set_path(processed_data_dir, 'ML-MATT-CompetitionQT1920_train_processed.parquet')
test_path = set_path(processed_data_dir, 'ML-MATT-CompetitionQT1920_test_processed.parquet')

### Load data

In [7]:
train_df = spark.read.parquet(train_path)
test_df = spark.read.parquet(test_path)

In [8]:
train_df.printSchema()

root
 |-- CellName: string (nullable = true)
 |-- hour: string (nullable = true)
 |-- minutes: string (nullable = true)
 |-- PRBUsageUL: double (nullable = true)
 |-- PRBUsageDL: double (nullable = true)
 |-- meanThr_DL: double (nullable = true)
 |-- meanThr_UL: double (nullable = true)
 |-- maxThr_DL: double (nullable = true)
 |-- maxThr_UL: double (nullable = true)
 |-- meanUE_DL: double (nullable = true)
 |-- meanUE_UL: double (nullable = true)
 |-- maxUE_DL: double (nullable = true)
 |-- maxUE_UL: double (nullable = true)
 |-- Unusual: integer (nullable = true)



In [9]:
train_df.show(5)

+--------+----+-------+----------+----------+----------+----------+---------+---------+---------+---------+--------+--------+-------+
|CellName|hour|minutes|PRBUsageUL|PRBUsageDL|meanThr_DL|meanThr_UL|maxThr_DL|maxThr_UL|meanUE_DL|meanUE_UL|maxUE_DL|maxUE_UL|Unusual|
+--------+----+-------+----------+----------+----------+----------+---------+---------+---------+---------+--------+--------+-------+
|   3BLTE|  10|     45|    11.642|     1.393|      0.37|     0.041|   15.655|    0.644|    1.114|    1.025|     4.0|     3.0|      1|
|   1BLTE|  09|     45|    21.791|     1.891|     0.537|     0.268|   10.273|    1.154|    1.353|    1.085|     6.0|     4.0|      1|
|   9BLTE|  07|     45|     0.498|     0.398|     0.015|      0.01|    0.262|    0.164|    0.995|    0.995|     1.0|     1.0|      1|
|   4ALTE|  02|     45|     1.891|     1.095|      0.94|     0.024|   60.715|    0.825|    1.035|    0.995|     2.0|     2.0|      1|
|  10BLTE|  03|     30|     0.303|     0.404|     0.016|     0

# 2. Machine Learning Pipeline

According to this scenario, our ML pipeline is composed by 4 stages:
- stage 1: String Indexing the "hour" variable
- stage 2: One Hot Encoding the indexed column of "hour" variable
- stage 3: Vectorizing all the features for MinMax standardization
- stage 4: Standardizing all variables
- stage 5: Train a GBTClassifier model

Let's build it!


## Train

In [16]:
stageOne = StringIndexer(inputCol='hour', 
                         outputCol='hour_index')

stageTwo = OneHotEncoder(dropLast=False, 
                         inputCol=stageOne.getOutputCol(), outputCol='hour_vec')

stageThree = VectorAssembler(
    inputCols=['PRBUsageUL', 'PRBUsageDL', 'meanThr_DL',
               'meanThr_UL', 'maxThr_DL', 'maxThr_UL', 
               'meanUE_DL', 'meanUE_UL', 'maxUE_DL','maxUE_UL', 'hour_vec'], 
    outputCol='vars_vectorized')

stageFour = MinMaxScaler(inputCol=stageThree.getOutputCol(), 
                         outputCol='features')

stageFive = GBTClassifier(featuresCol=stageFour.getOutputCol(), labelCol='Unusual', maxIter=3, seed=888)

In [17]:
gbt_pipeline = Pipeline(stages=[stageOne, stageTwo, stageThree, stageFour, stageFive])

In [18]:
gbt_model = gbt_pipeline.fit(train_df)

In [19]:
predictions_train= gbt_model.transform(train_df)

In [21]:
predictions_train.select('CellName', 'features', 'Unusual', 'rawPrediction', 'probability', 'prediction').show(5)

+--------+--------------------+-------+--------------------+--------------------+----------+
|CellName|            features|Unusual|       rawPrediction|         probability|prediction|
+--------+--------------------+-------+--------------------+--------------------+----------+
|   3BLTE|(34,[0,1,2,3,4,5,...|      1|[0.51677462414346...|[0.73760341280871...|       0.0|
|   1BLTE|(34,[0,1,2,3,4,5,...|      1|[0.79496718396118...|[0.83060686889221...|       0.0|
|   9BLTE|(34,[0,1,2,3,4,5,...|      1|[-1.0915005269131...|[0.10128742007114...|       1.0|
|   4ALTE|(34,[0,1,2,3,4,5,...|      1|[-0.9762705534631...|[0.12427654059452...|       1.0|
|  10BLTE|(34,[0,1,2,3,4,5,...|      0|[1.09150052691311...|[0.89871257992884...|       0.0|
+--------+--------------------+-------+--------------------+--------------------+----------+
only showing top 5 rows



## Test

In [23]:
predictions_test= gbt_model.transform(test_df)
predictions_test.select('CellName', 'features', 'rawPrediction', 'probability', 'prediction').show(5)

+--------+--------------------+--------------------+--------------------+----------+
|CellName|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+--------------------+--------------------+----------+
|   6ALTE|(34,[0,1,2,3,4,5,...|[-0.0619848391774...|[0.46904721170647...|       1.0|
|   6ULTE|(34,[0,1,2,3,4,5,...|[0.90087721285987...|[0.85836236602181...|       0.0|
|   2ALTE|(34,[0,1,2,3,4,5,...|[0.92592783763985...|[0.86434483333830...|       0.0|
|   3CLTE|(34,[0,1,2,3,4,5,...|[1.10984499743149...|[0.90200379684945...|       0.0|
|   6CLTE|(34,[0,1,2,3,4,5,...|[-0.2149457119780...|[0.39415225882297...|       1.0|
+--------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



## Serialize the pipeline

In [28]:
gbt_pipeline.write().overwrite().save(path=deliverables_path)

# Conclusion

We get our Pipeline. Let's version it in SAS Model Manager